<a href="https://colab.research.google.com/github/daniyar-aussabayev/Colabs-for-sharing/blob/main/My_ToDo_to_JSC_migration_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [ ]:
import pandas as pd
import re
import os
from IPython.display import display, Markdown
def print_markdown(text):
  display(Markdown(text))
answer = "Yes"

# Please upload your file

# Select the file

In [ ]:
#@markdown
available_files = [f for f in os.listdir() if '.csv' in f]

if answer == "Yes" and len(available_files) > 0:
  print_markdown('## 📁 List of Available csv files:')
  print('\n')
  for num,filename in enumerate(available_files):
    # print_markdown(f"**{num + 1})** **{filename}**")
    print_markdown(f'**{filename}**')
    print_markdown('---')
elif answer == "Yes" and len(available_files) == 0:
  print_markdown('## No csv files were found. Please upload the file and re-run this cell')
else:
  print_markdown('## Please upload the file first and re-run this cell')

In [ ]:
if answer == "Yes":
  #@markdown ### Please paste the filename here from the list above
  filename = "UHUBBIZ-3128 (1).csv" #@param {type:"string"}
  try:
    df = pd.read_csv(filename)
    df['Custom field (To Do List)'] = df['Custom field (To Do List)'].str.replace('null','None')#.str.replace('""','"', regexp=True)
  except:
    print(f'Could not read the file {filename}')
else:
  print_markdown('## Please upload the file first and re-run this cell')

# Preparation of output file

In [ ]:
#@markdown
finished = False
if answer == "Yes":
  statuses_dict = {'done':'+',
                  'todo_undone':'-',
                  'in_progress':'~',
                  'cancelled':'x',
                   'todo':'-'}

  jsc_df = pd.DataFrame()

  for each_task in df.index:
      jsc_df.loc[each_task,'Issue key'] = df.loc[each_task,'Issue key']
      jsc_df.loc[each_task,'Summary'] = df.loc[each_task,'Summary']
      try:
        each_task_d = eval(df.loc[each_task,'Custom field (To Do List)'])
      except:
        try:
          each_task_d = eval(df.loc[each_task,'Custom field (To Do List)'].replace('"+', '"').replace('"X"','\"X\"'))
        except:
          each_task_d = [{'id':'','type':''}]
          print(each_task,df.loc[each_task,'Custom field (To Do List)'])
      checklist = []
      for item in each_task_d:
          status = statuses_dict.get(item['type'])
          checklist_item = item['id']
          links = re.findall('\[.+\|.+\]',checklist_item)
          old_link = ''.join(links)
          new_link = ''
          for link in links:
              new_link = f"[{link.split('|')[0][:-1][1:]}]({link.split('|')[1][:-1]})"
              checklist_item = checklist_item.replace(link,new_link)
          
          user_mentions = re.findall('\[~.*?\]',checklist_item)
          new_user_mention = ''
          for user_mention in user_mentions:
              new_user_mention = f"[@{user_mention[2:-1]}]"
              checklist_item = checklist_item.replace(user_mention,new_user_mention)
              
          checklist.append(f'{status} {checklist_item}')
      
      checklist = '\n'.join(checklist)
      jsc_df.loc[each_task,'Checklist'] = checklist
  jsc_df.to_csv('Prepared checklist.csv', index=False, sep=',')
  finished = True
else:
  print_markdown('## Please upload the file first')

#@markdown
if finished:
  print_markdown('## ✅ Modified file will appear, it is called **Prepared checklist.csv**')
  print()
  print_markdown('Once it appears Download it by right-clicking on it and selecting **Download**')
  print_markdown('![image.png](...)')